In [1]:
import pandas as pd
import numpy as np
import nltk
import math
import time
import operator
import string
from sklearn.cluster import KMeans
from math import pi
from sklearn.metrics import mean_squared_error
from nltk.corpus import wordnet, stopwords, sentiwordnet as swn
from nltk.tokenize import sent_tokenize, word_tokenize, WhitespaceTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
f = open('test_data.csv','r', errors='ignore')
test_data = pd.read_csv(f,
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True)

test_data.head(30)


,asin,date,downvotes,review_comments,review_id,review_text,review_type,reviewer,upvotes,categories,...,in_app_purchase,mas_rating,min_os_version,num_permissions,num_screenshots,price,release_date,size_MB,title,update_date
0,B00IZJLRQE,2014-07-15,NaN,0,R1EZKJDJ5UAF5O,Best game ever,Verified Purchase,A30JKGN6OEFHT,NaN,Games,...,0,All Ages,2.2,5,9,99.0,2014-03-14,21.900000,Flappy Nyan - Ad Free,2014-06-11
1,B00ET0PZDO,2014-05-04,NaN,0,RNVH13QG7XY8Q,this is a very fun and challenging game we ne...,Verified Purchase,A3KBSK3ASRB8V4,NaN,Games,...,0,All Ages,2.2,6,9,99.0,2013-08-29,54.200000,You Must Escape,2014-07-17
2,B004N945WS,2015-04-21,NaN,0,R2R90UNBEGV0MP,"Good app, if you like to focus on exercise.......",Verified Purchase,AUJ7TYMUAOAIH,NaN,"Lifestyle,Health & Fitness",...,0,All Ages,1.6,3,8,194.0,2010-10-31,1.000000,Diet Assistant Pro,2012-02-06
3,B00CGPPMKU,2013-07-02,NaN,0,R11XZ8I2SEK3PC,If I could give it more stars I would!Vast col...,Verified Purchase,A38DIZ8ADMXF4M,NaN,Novelty,...,1,All Ages,2.2,3,7,199.0,2013-04-25,4.100000,Flowpaper,2013-07-08
4,B00C5WZ2L8,2013-08-28,NaN,0,RLUVG914XCDG8,This game really keeps you thinking. Lets you ...,Verified Purchase,A3VDKNF4C2K0MC,NaN,Games,...,1,All Ages,2.3.3,9,6,NaN,2013-04-05,54.800000,4Pics 1Word- What's the Word Init,2014-10-31
5,B004T2N9VC,2013-01-03,NaN,0,R2884NHGQCOHF3,the app is password protected and it works gre...,Verified Purchase,AV1B51Q631PCX,NaN,"Productivity,Utilities",...,0,All Ages,1.5,3,4,100.0,2011-03-16,0.131055,PassDiary - The Password Manager,2011-11-01
6,B007SVPZMW,2013-10-20,NaN,0,R2JMYK0QZPCI9H,I got this for the FAOTD and thought my 3 year...,Verified Purchase,A28VS5BVSLTAWD,NaN,Games,...,0,All Ages,2.1,1,6,99.0,2012-03-30,25.300000,Terrific Tic Tac Toe HD,2014-01-28
7,B00AB7HESI,2016-01-17,NaN,0,R3H5JIN6M9OA1J,Awesome game! I have scribblenauts for DSi and...,Verified Purchase,A2TIL2YJWWJOJA,NaN,Games,...,1,Guidance Suggested,2.3.3,14,6,99.0,2012-12-10,179.500000,Scribblenauts Remix,2016-04-29
8,B009ZKSPDK,2013-11-13,NaN,0,R38LDR54VV3AW9,My son is having a great time with this game. ...,Verified Purchase,A39TPD0FW7O2PE,NaN,Games,...,0,Guidance Suggested,2.3.3,6,9,399.0,2012-11-06,16.900000,Survivalcraft,2016-03-16
9,B0093QQAEM,2014-12-07,NaN,0,R2Y9Z7VCO2M055,Only old old old movies,Verified Purchase,A35EYNTV061Q72,NaN,Movies & TV,...,0,Guidance Suggested,4.0.3,4,7,99.0,2012-08-31,5.500000,Free Movies Now Pro,2015-12-26


In [18]:
# test_review = [(list(test_data.review_text(asin1)),reviewid)
#               for reviewid in test_data.review_id()
#                for asin1 in test_data.asin(reviewid)
#               ]

#start = time.time()

#word_score_pos_list = [None]*len(test_data1)
#word_score_neg_list = [None]*len(test_data1)
test_review = pd.Series.tolist(test_data.review_text)
test_review = [str(tmp) for tmp in test_review]
length = len(test_review)
#test_review = [None]*length
# for i in range(length):
# #     if test_data1[i] == None:
# #         test_data1[i] = the
#     test_data1[i] = str(test_data1[i])
#     test_review[i] = "".join([ch for ch in test_data1[i] if ch not in set(string.punctuation)])
score = np.zeros(length)
start = time.time()
piror_pos = 0.01
piror_neg = 0.01

test_voc = TfidfVectorizer(decode_error='ignore', ngram_range=(1,2), max_df=0.8, min_df=0.01, stop_words='english')

# for i in range(length):
#     for words in test_review:
        
for i in range(length):
    word_tokens = word_tokenize(test_review[i])
    word_score = 0
    word_score_pos1 = 0
    word_score_neg1 = 0
    for words in word_tokens:
        word_score_pos = 0
        word_score_neg = 0
        syns = wordnet.synsets(words)
        for t in range(len(syns)):
            #if word_score_pos < swn.senti_synset(syns[t].name()).pos_score():
            word_score_pos += swn.senti_synset(syns[t].name()).pos_score()
            #if word_score_neg < swn.senti_synset(syns[t].name()).neg_score():
            word_score_neg += swn.senti_synset(syns[t].name()).neg_score()
        word_score_pos1 += word_score_pos
        word_score_neg1 += word_score_neg
        #word_score_pos1 = word_score_pos1/len(word_tokens)
        #word_score_neg1 = word_score_neg1/len(word_tokens)
    score[i] = (word_score_pos1+piror_pos)/(word_score_neg1+piror_neg)
    #math.atan2(word_score_pos1 + piror_pos,word_score_neg1 + piror_neg)-pi/4
print(score)
end = time.time()
print(end - start)

[ 10.47634974   1.15852885  30.52755906 ...,   7.36603409   1.68072025
   0.85199803]
385.89309191703796


In [6]:
test_id = pd.Series.tolist(test_data.review_id)
id_score_list = dict(zip(test_id,score))
sorted_list = sorted(id_score_list.items(), key=operator.itemgetter(1))
print(score)
#print(sorted_list)

[ 0.69023339  0.07331139  0.75265259 ...,  0.65046482  0.24867579
 -0.07974528]


In [8]:
splitlist = list(zip(*sorted_list))
#print(splitlist)

In [9]:
#print(splitlist[1])
inkmean = [[tmp] for tmp in splitlist[1]]
#print(inkmean)
kmeans = KMeans(n_clusters = 5, init='k-means++')
kmeans.fit(inkmean)
centroids = kmeans.cluster_centers_

centroids1 = sorted(centroids)
print(centroids1)

[array([-0.3797216]), array([-0.04938387]), array([ 0.17621345]), array([ 0.39126844]), array([ 0.62807013])]


In [10]:
distance = np.zeros((5,100000))
score1 = [[tmp] for tmp in score]
for i in range(5):
    for j in range(100000):
        distance[i][j] = mean_squared_error(centroids1[i], score1[j])

In [13]:
prob = np.zeros((100000,5))
distance_t = 1/np.transpose(distance)
for i in range(100000):
    for j in range(5):
        prob[i][j] = (distance_t[i][j]/sum(distance_t[i]))

array([[  3.15968436e-03,   6.61242325e-03,   1.36903572e-02,
          4.04700893e-02,   9.36067446e-01],
       [  2.72383189e-02,   3.71350840e-01,   5.27948812e-01,
          5.52971859e-02,   1.81648428e-02],
       [  1.00717941e-02,   2.00769873e-02,   3.88668946e-02,
          9.88891340e-02,   8.32095190e-01],
       ..., 
       [  4.67331350e-04,   1.01262409e-03,   2.20515626e-03,
          7.38241922e-03,   9.88932469e-01],
       [  9.72630641e-03,   4.32324946e-02,   7.31462334e-01,
          1.88895844e-01,   2.66830205e-02],
       [  9.94265608e-03,   9.70582298e-01,   1.36564074e-02,
          4.03282370e-03,   1.78581442e-03]])

In [15]:
prob1 = prob.astype('str')
review_id = pd.Series.tolist(test_data.review_id)
review_id = np.asarray([[str(tmp)] for tmp in review_id])
result = np.append(review_id, prob1, 1)
title = np.asarray([['review_id','1-star','2-star','3-star','4-star','5-star']])
result = np.append(title,result,0)
#result = [np.array(tmp) for tmp in result]
result = result.tolist()
result = np.asarray(result)
np.savetxt("result1.csv",result,fmt='%s,%s,%s,%s,%s,%s')